In [1]:
# Download dataset
!wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/subjectivity/subjectivity-negative-bm.txt
!wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/subjectivity/subjectivity-positive-bm.txt

--2019-09-11 02:50:22--  https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/subjectivity/subjectivity-negative-bm.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 700425 (684K) [text/plain]
Saving to: ‘subjectivity-negative-bm.txt’

subjectivity-negati 100%[===================>] 684.01K  --.-KB/s    in 0.06s   

2019-09-11 02:50:22 (10.7 MB/s) - ‘subjectivity-negative-bm.txt’ saved [700425/700425]

--2019-09-11 02:50:24--  https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/subjectivity/subjectivity-positive-bm.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connect

In [2]:
# Check if have the dataset downloaded
!ls

sample_data  subjectivity-negative-bm.txt  subjectivity-positive-bm.txt


In [0]:
with open('subjectivity-negative-bm.txt') as fopen:
  negative = fopen.read().split('\n')[:-1]

with open('subjectivity-positive-bm.txt') as fopen:
  positive = fopen.read().split('\n')[:-1]

In [4]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1270525559738965315, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5407650109671801694
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12025918761234603702
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5870716687150579958
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [5]:
# Check the size
len(negative), len(positive)

# Pre-process data
import re

def cleaning(string):
  string = re.sub('[^A-Za-z ]+', ' ', string.lower())
  string = re.sub(r'[ ]+', ' ', string).strip()
  return string

cleaning(negative[0])


for i in range(len(negative)):
  negative[i] = cleaning(negative[i])

for i in range(len(positive)):
  positive[i] = cleaning(positive[i])

X = negative + positive
Y = [0] * len(negative) + [1] * len(positive)
len(X), len(Y)


(9960, 9960)

In [6]:
X[0]

'filem bermula pada masa lalu di mana seorang budak lelaki bernama sam cuba menyelamatkan celebi dari pemburu'

In [0]:
import collections
import json

def build_dataset(words, n_words, atleast=1):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reversed_dictionary

In [0]:
concat = ' '.join(X).split()
size_concat = len(set(concat))
dictionary, reversed_dictionary = build_dataset(concat, size_concat, atleast=1)

In [9]:
X[0]

'filem bermula pada masa lalu di mana seorang budak lelaki bernama sam cuba menyelamatkan celebi dari pemburu'

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
class Model:
  def __init__(self, size_layer, num_layers, learning_rate, class_size):
    
    self.X = tf.placeholder(tf.int32, (None, None))
    self.Y = tf.placeholder(tf.int32, (None))
    
    # word vector definition
    embeddings = tf.Variable(tf.random_uniform([len(dictionary), size_layer], -1, 1))
    
    # lookup batch of integers to word vector
    embedded = tf.nn.embedding_lookup(embeddings, self.X)
    
    def cells(reuse=False):
      return tf.nn.rnn_cell.LSTMCell(size_layer,
                                     initializer=tf.orthogonal_initializer(),
                                     reuse=reuse)
    
    # in tensorflow, we need to create function that return cell to build multirnn
    rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
    
    # output size [B, T, D]
    output, _ = tf.nn.dynamic_rnn(rnn_cells, embedded, dtype = tf.float32)
    
    # logits size [B, D]
    self.logits = tf.layers.dense(output[:,-1], class_size)
    self.cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.logits, 
                                                               labels = self.Y)
    self.cost = tf.reduce_mean(self.cost)
    
    self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
    
    correct_pred = tf.equal(
        tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
    )
    self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [16]:
size_layer = 256
num_layers = 2
learning_rate = 1e-3
class_size = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer, num_layers, learning_rate, class_size)
sess.run(tf.global_variables_initializer())

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
def str_idx(corpus, dic, maxlen, UNK = 3):
    X = np.zeros((len(corpus), maxlen))
    for i in range(len(corpus)):
        for no, k in enumerate(corpus[i][:maxlen][::-1]):
            X[i, -1 - no] = dic.get(k, UNK)
    return X

In [0]:
maxlen = 100
X_idx = str_idx(X, dictionary, maxlen)

In [0]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(X_idx, Y, test_size = 0.2)

batch_size = 128
epoch = 10
PAD = 0

In [20]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i : index]
        batch_y = train_Y[i : index]
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
        
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i : index]
        batch_y = test_Y[i : index]
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('\nepoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('\nepoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   2%|▏         | 1/63 [00:00<00:11,  5.22it/s, accuracy=0.531, cost=0.697]


epoch 1, training avg loss 0.696885, training avg acc 0.517485

epoch 1, testing avg loss 0.686804, testing avg acc 0.531467


minibatch loop:   2%|▏         | 1/63 [00:00<00:11,  5.21it/s, accuracy=0.594, cost=0.699]


epoch 2, training avg loss 0.680036, training avg acc 0.568700

epoch 2, testing avg loss 0.684244, testing avg acc 0.597439


minibatch loop:   2%|▏         | 1/63 [00:00<00:12,  5.14it/s, accuracy=0.578, cost=0.689]

...


epoch 8, training avg loss 0.468670, training avg acc 0.788690

epoch 8, testing avg loss 0.512206, testing avg acc 0.770671


minibatch loop:   0%|          | 0/63 [00:00<?, ?it/s]


epoch 9, training avg loss 0.414847, training avg acc 0.814608

epoch 9, testing avg loss 0.504113, testing avg acc 0.780816


minibatch loop: 100%|██████████| 16/16 [00:01<00:00, 11.82it/s, accuracy=0.792, cost=0.468]


epoch 10, training avg loss 0.395789, training avg acc 0.823165

epoch 10, testing avg loss 0.544042, testing avg acc 0.780924
